In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
from IPython.display import Image

/var/folders/hv/b1wcvr5j7bb1mzjm00fwl5zm0000gn/T/ipykernel_20020/361487260.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# fetch the PhishingVSBenignURL dataset
file_path = '/Users/nisha/Desktop/spring2024/MachineLearning/DataSetForPhishingVSBenignUrl.csv'
df = pd.read_csv(file_path)

df.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


In [3]:
# encode target column "URL_Type_obf_Type" using LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df['URL_Type_obf_Type'])
encoded_label = label_encoder.transform(df['URL_Type_obf_Type'])

# replace old "URL_Type_obf_Type" with encoded_label
feature_name = ['URL_Type_obf_Type']
encoded_df = pd.DataFrame(encoded_label, columns=feature_name)
df_encoded = pd.concat([df.drop('URL_Type_obf_Type', axis=1), encoded_df], axis=1)

print(df_encoded.head())

   Querylength  domain_token_count  path_token_count  avgdomaintokenlen  \
0            0                   4                 5                5.5   
1            0                   4                 5                5.5   
2            0                   4                 5                5.5   
3            0                   4                12                5.5   
4            0                   4                 6                5.5   

   longdomaintokenlen  avgpathtokenlen  tld  charcompvowels  charcompace  \
0                  14         4.400000    4               8            3   
1                  14         6.000000    4              12            4   
2                  14         5.800000    4              12            5   
3                  14         5.500000    4              32           16   
4                  14         7.333334    4              18           11   

   ldl_url  ...  SymbolCount_FileName  SymbolCount_Extension  \
0        0  ...             

In [4]:
# create X and y
X = df_encoded.iloc[:, :-1]
y = df_encoded.iloc[:, -1]

In [5]:
# check for infinite values in X
has_infinite = np.isinf(X)
if np.any(has_infinite):
    print("There are infinite values in the dataset.")
else:
    print("There are no infinite values in the dataset.")

There are infinite values in the dataset.


In [6]:
# sklearn's DecisionTreeClassifier can't work w/ infinite values
    # find exact location of infinite values
rows_with_infinite = np.any(has_infinite, axis=1)
cols_with_infinite = np.any(has_infinite, axis=0)

print(f"Rows with infinite values: {np.where(rows_with_infinite)[0]}")
print(f"Columns with infinite values: {np.where(cols_with_infinite)[0]}")

Rows with infinite values: [22422 23183 26468 27804 28029 28435 29619 29766 29944 29995]
Columns with infinite values: [31]


In [7]:
# replace infinite values with NaN
    # note: Column 31 = 'argPathRatio'
arr = np.where(rows_with_infinite)[0]
X.loc[arr, 'argPathRatio'] = np.nan

In [8]:
# split X and y into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# train decision tree classifier on all features
tree = DecisionTreeClassifier(random_state=50)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=50)

In [10]:
# train DecisionTreeClassifier for depth 1, 2, 3, 4, 5, 6
    # train for Gini and Entropy impurity measures at each depth
accuracy = []
accuracy.append(['Impurity Measure', 'Depth', 'Train Accuracy', 'Test Accuracy'])
impurity_measures = ['gini', 'entropy']
for i in range(1, 7):
    for j in impurity_measures:
        tree = DecisionTreeClassifier(criterion=j, max_depth=i, random_state=50)
        tree.fit(X_train, y_train)
        y_pred_train = tree.predict(X_train)
        y_pred_test = tree.predict(X_test)
        train_accuracy = accuracy_score(y_train, y_pred_train)
        test_accuracy = accuracy_score(y_test, y_pred_test)
        curr_acc = [j, i, train_accuracy, test_accuracy]
        accuracy.append(curr_acc)

In [11]:
# compare results of DecisionTrees
accuracy_df = pd.DataFrame(accuracy)
print(accuracy_df)

                   0      1               2              3
0   Impurity Measure  Depth  Train Accuracy  Test Accuracy
1               gini      1        0.377218       0.375511
2            entropy      1        0.377218       0.375511
3               gini      2          0.4945       0.493326
4            entropy      2        0.495249       0.492373
5               gini      3        0.484727        0.48243
6            entropy      3        0.592712       0.586897
7               gini      4        0.675192       0.671343
8            entropy      4        0.669947       0.665622
9               gini      5        0.730598       0.730046
10           entropy      5        0.728214       0.728139
11              gini      6        0.754163       0.753609
12           entropy      6        0.737238       0.742713


### The decision tree with depth 6 and impurity measure Gini had he highest train and test accuracy (~75%).

In [12]:
# retrain best performing tree and visualize it
best_tree = DecisionTreeClassifier(criterion='gini', max_depth=6, random_state=50)
best_tree.fit(X_train, y_train)

export_graphviz(best_tree, out_file='tree.dot')

In [13]:
!dot -Tpng tree.dot -o tree.png -Gdpi=1000

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.226702 to fit


In [14]:
Image(filename = 'tree.png')

### One observation I made is that many of the leaf nodes still have quite a high measure of impurtiy (gini ~= .5 or higher). Additionally, I observed that this tree is very complex. There are many splits. This indicates that the classification task attempted by this model is very complex. Finally, I observed that this model makes use of several different features for the many splits it has, indicating that many features of the original dataset have an impact on this classification task.